# Aprendizaje Supervizado - Supervivencia en el Titanic

## Extracción de Características

En esta parte se trabajará con extracción de características cuantitativas a partir de los datos.

- Se extraeran características a partir de datos del mundo real.
- Extraer características numéricas a partir de datos textuales.

### Características

**Características Numéricas**: 
Hasta el momento se han trabajado con datos (datasets) de scikit-learn que se presentan como arrays con tamaño **n_muestras** x **m_features**, como por ejemplo el dataset de Iris, con 150 muestras y 4 características.

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
print(iris.data.shape)

(150, 4)


- Características
    - Longitud del sépalo [cm]
    - Ancho del sépalo [cm]
    - Longitud del pétalo [cm]
    - Ancho del pépalo [cm]

**Características Categóricas**: Imagine que tiene disponible el color de cada flor de Iris: $color \in \{red, blue, purple\}$, en donde se podría pensar en trabajar con *red=1, blue=2, purple=3*, pero no es recomendable debido a que los estimadores tienden a trabajar con la suposición de que las características numéricas se sitúan en una escala continua. Por ejemplo, 1 y 2 (colores red y blue) serían más parecidos que 1 y 3 (colores red y purple), y esto no tiene mucho sentido, al menos en este ejemplo.

<div class="alert alert-success">
    Este ejemplo es una subcategoria de las variables categóricas denominada <b>variable nominal</b>. Las variables nominales no tienen asociado un orden, mientras que las <b>variables ordinales</b> si lo implican, por ejemplo, las tallas de las camisetas se considera una variable ordinal XL > L > M > S.
</div>

Para transformar variables nominales en un formato que prevenga al estimador de asumir un orden es usando la representación **One-hot Encoding**, en donde cada categoría genera su propia variable por separado.

- Características
    - Longitud del sépalo [cm]
    - Ancho del sépalo [cm]
    - Longitud del pétalo [cm]
    - Ancho del pépalo [cm]
    - color=purple (1.0 o 0.0)
    - color=blue (1.0 o 0.0)
    - color=red (1.0 o 0.0)

<div class="alert alert-warning">
    Es muy probable que debido a este tipo de características sea mejor usar matrices dispersas.
</div>

- **DictVectorizer**

La clase DictVectorizer es útil cuando los datos de entrada se nos presentan como un diccionario con valores que son cadenas o valores numéricos y se quiere obtener una expansión booleana.

Por ejemplo:

In [2]:
medidas = [
    {'city': 'Dubai', 'temperature': 33.},
    {'city': 'London', 'temperature': 12.},
    {'city': 'San Francisco', 'temperature': 18.}
]

In [3]:
medidas

[{'city': 'Dubai', 'temperature': 33.0},
 {'city': 'London', 'temperature': 12.0},
 {'city': 'San Francisco', 'temperature': 18.0}]

In [6]:
from sklearn.feature_extraction import DictVectorizer

vector = DictVectorizer()
vector.fit_transform(medidas).toarray()

array([[ 1.,  0.,  0., 33.],
       [ 0.,  1.,  0., 12.],
       [ 0.,  0.,  1., 18.]])

In [7]:
vector.get_feature_names_out()

array(['city=Dubai', 'city=London', 'city=San Francisco', 'temperature'],
      dtype=object)

**Características Derivadas**: Estas son obtenidas a partir de algún paso previo de preprocesamiento y que se suponen aportan mayor información al estimador. Puede obtenerse mediante **extración de características** o por **reducción de dimensionalidad** ó pueden ser combinaciones lineales o no lineales de características originales o alguna otra transformación.

## Supervivencia en el Titanic
### Características Numéricas y Categóricas

En el ejemplo a continuación se tratará de predecir la supervicencia de los pasajeros del HMS Titanic. Se trabajará con el siguiente dataset [titanic3.csv](https://raw.githubusercontent.com/amueller/scipy-2017-sklearn/master/notebooks/datasets/titanic3.csv).

Para trabajar con este archivo se debe:

- Leer todas las líneas ignorando la cabecera. 
- Leer los datos de entrada - (Características de la persona)
- Encontrar las etiquetas (sobrevivió - murió)

In [9]:
import os
import pandas as pd

titanic = pd.read_csv(os.path.join('datasets', 'titanic3.csv'))
# datasets/titanic3.csv
type(titanic)

pandas.core.frame.DataFrame

In [10]:
titanic.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


Esta es una descripcion de cada una de las columnas (características)

```

pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
survival        Survival
                (0 = No; 1 = Yes)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)
boat            Lifeboat
body            Body Identification Number
home.dest       Home/Destination

```

Al analizar el dataset se puede observar que **name, sex, cabin, embarked, boat, body y home/dest** son candidatas a variables categoricas, mientras que el resto parecen variables numéricas. Observe los primeros datos del dataset:

In [11]:
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


Para este ejercicio de predecir la sobrevivencia de los pasajeros se debe descartar **boat** y **body** ya que nos informa lo que sucedio con ellos. EL nombre es único para cada persona y no está relacionado con su sobrevivienvia.

Se usarán las siguientes características:

- pclass
- sex
- age
- sibsp
- parch
- fare
- embarked


In [12]:
etiquetas = titanic.survived.values
caracteristicas = titanic[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

In [15]:
caracteristicas.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,1,female,29.0000,0,0,211.3375,S
1,1,male,0.9167,1,2,151.5500,S
2,1,female,2.0000,1,2,151.5500,S
3,1,male,30.0000,1,2,151.5500,S
4,1,female,25.0000,1,2,151.5500,S


Por el momento, suponemos que los datos que se han extraído son características útiles para nuestro estimador. Sin embargo, estos datos no estan en un formato que los algortimos de Machine Learning puedan entender. La características **sex** y **embark** deben ser transformadas en variables binarias. Para ello se usará el método **get_dummies** de pandas.

In [17]:
pd.get_dummies(caracteristicas).head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,1,29.0000,0,0,211.3375,1,0,0,0,1
1,1,0.9167,1,2,151.5500,0,1,0,0,1
2,1,2.0000,1,2,151.5500,1,0,0,0,1
3,1,30.0000,1,2,151.5500,0,1,0,0,1
4,1,25.0000,1,2,151.5500,1,0,0,0,1


Si se observa las últimas columnas se ve que la transformación se ha realizado con éxito. Sin embargo, se ha ignorado la variable **pclass** que también es categórica.

El método **get_dummies** permite listar de forma explícita las variables que se quieren codificar mediante el parámetro **columns**.

In [18]:
caracteristicas_dummies = pd.get_dummies(caracteristicas, columns=['pclass', 'sex', 'embarked'])
caracteristicas_dummies.head()

,age,sibsp,parch,fare,pclass_1,pclass_2,pclass_3,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,29.0000,0,0,211.3375,1,0,0,1,0,0,0,1
1,0.9167,1,2,151.5500,1,0,0,0,1,0,0,1
2,2.0000,1,2,151.5500,1,0,0,1,0,0,0,1
3,30.0000,1,2,151.5500,1,0,0,0,1,0,0,1
4,25.0000,1,2,151.5500,1,0,0,1,0,0,0,1



También es posible realizar esta tarea con **DictVectorizer**


In [20]:
diccionario = caracteristicas.to_dict('records')
vec = DictVectorizer()
dataset = vec.fit_transform(diccionario)
print(type(dataset))
print(dataset.todense())

<class 'scipy.sparse._csr.csr_matrix'>
[[29.      0.      0.     ...  1.      0.      0.    ]
 [ 0.9167  0.      0.     ...  0.      1.      1.    ]
 [ 2.      0.      0.     ...  1.      0.      1.    ]
 ...
 [26.5     0.      1.     ...  0.      1.      0.    ]
 [27.      0.      1.     ...  0.      1.      0.    ]
 [29.      0.      0.     ...  0.      1.      0.    ]]


In [21]:
# Covertir el dataframe en array de numpy
print(type(caracteristicas_dummies))
data = caracteristicas_dummies.values
type(data)

<class 'pandas.core.frame.DataFrame'>


numpy.ndarray

In [23]:
# Comprabar que no existan valores perdidos NaN
import numpy as np
np.isnan(data).any()

# Si es True contiene Nan

True

Hasta esta parte se ha cargado los datos, filtrado y construido el dataset. Ahora se eligirá un clasificador. Para este ejercicio se usará **DummyClassifier('most_frequent')**.

<div class="alert alert-success">
    DummyClassifier hace predicciones que ignoran las características de entrada. Este clasificador sirve como una línea de base simple para comparar con otros clasificadores más complejos. El comportamiento específico de la línea de base se selecciona con el parámetro de estrategia. Todas las estrategias hacen predicciones que ignoran los valores de características de entrada pasados como argumento <b>X</b> para ajustar y predecir. Las predicciones, sin embargo, generalmente dependen de los valores observados en el parámetro <b>y</b> pasado para ajustar.
</div>

Como los datos presentan **Nan** se usará **SimpleImputer** para corregir este problema.

<div class="alert alert-success">
    Imputador univariante para completar valores faltantes con estrategias simples. Reemplace los valores faltantes usando una estadística descriptiva (por ejemplo, media, mediana o más frecuente) a lo largo de cada columna, o usando un valor constante.
<div>

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

train_data, test_data, train_labels, test_labels = train_test_split(data, etiquetas, random_state=0)

imp = SimpleImputer()
imp.fit(train_data)
train_data_finite = imp.transform(train_data)
test_data_finite = imp.transform(test_data)

In [27]:
# Volvemos apreguntar si existen valores Nan
np.isnan(train_data_finite).any()

False

In [28]:
np.isnan(test_data_finite).any()

False

In [29]:
from sklearn.dummy import DummyClassifier

dmc = DummyClassifier(strategy='most_frequent')
dmc.fit(train_data_finite, train_labels)
print(f"Accuracy: {dmc.score(test_data_finite, test_labels)}")

Accuracy: 0.6341463414634146


<div class="alert alert-info">
    <b>EJERCICIO:</b>
    <ul>
        <li>Usar LogisticRegression y RandomForestClassifier</li>
        <li>Probar con otras características - Se consiguen mejores resultados?</li>
    </ul>
</div>

[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)

**from sklearn.linear_model import LogisticRegression**

[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

**from sklearn.ensemble import RandomForestClassifier**

In [41]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(max_iter=1000)
lg.fit(train_data_finite, train_labels)
lg.score(test_data_finite, test_labels)

0.7926829268292683

In [52]:
from sklearn.ensemble import RandomForestClassifier

rdf = RandomForestClassifier(n_estimators=200, oob_score=False, n_jobs=2, random_state =0)
rdf.fit(train_data_finite, train_labels)
rdf.score(test_data_finite, test_labels)

0.7774390243902439